In [ ]:
from nbdev import *

In [ ]:
#| hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

# Baseline Models

> This notebook tries generate baseline models to evaluate if DCAE models have a good performance or nor


In [ ]:
#| hide
from fastcore import test
import pandas as pd
import numpy as np

In [ ]:
#| hide
from timecluster_extension.load import *
from timecluster_extension.dr import *
from timecluster_extension.visualization import *
from timecluster_extension.utils import *
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbCallback
from yaml import load, FullLoader
from fastcore.utils import Path
from datetime import datetime
import pickle

Create a run to save the models (job_type = "baseline_models")

In [ ]:
#| hide
run_baseline = wandb.init(entity = "pacmel",
                      project="timecluster-extension",
                      job_type='baseline_models',
                      allow_val_change=True,
                      resume=False)
config = wandb.config  # Object for storing hyperparameters

## Load the datasets

In [ ]:
artifact_name_and_version = 'JNK:train_10days'

In [ ]:
ds_train_artifact = run_baseline.use_artifact(artifact_name_and_version)

In [ ]:
# parameters (uncomment to override the yaml file)
config.update(
    {
          'ds_train_artifact_type': ds_train_artifact.type,
          'ds_train_artifact_name': ds_train_artifact.name,
          'ds_train_artifact_digest': ds_train_artifact.digest,
    }, 
    allow_val_change=True)
ds_train_artifact.type, ds_train_artifact.name, ds_train_artifact.digest

In [ ]:
df_train = ds_train_artifact.to_df()

In [ ]:
df_train.head(1)

## Train

### Sliding window features

In [ ]:
config.w = ifnone(config.get('w'), 48)
config.stride = ifnone(config.get('stride'), 1)
config.t = ifnone(config.get('t'), 0)

In [ ]:
# parameters (uncomment to override the yaml file)
config.update({
    'w': config.w,
    'stride': config.stride,
    't': config.t  # TODO: Not supported yet
    }, allow_val_change=True)

In [ ]:
test.equals(config.w % 12, 0)

In [ ]:
input_data = df_slicer(df_train, w=config.w, s=config.stride)

In [ ]:
data_test = input_data#Take 10 windows data_test[0:10,:,:]

In [ ]:
data_test.shape

In [ ]:
#| export utils
def baseline_model_predictor(input_array, operation = "mean"):
    " Perform an arithmetic operation (median or average) on a three-dimensional numpy array from df_slicer"
    # Calculate mean/median for each window in the dataset
    if operation == "mean":
        prediction = np.mean(input_array,axis=1)
    elif operation == "median":
        prediction = np.median(input_array,axis=1)
    # Generate an output numpy array with the same size that input_array with
    # baseline predictions
    # Create a 3-d numpy array with ones
    output_array = np.ones(input_array.shape)
    # Multiply it by the prediction array, with a new dimension in axis 1
    output_array = output_array * np.expand_dims(prediction, axis=1)
    return output_array

In [ ]:
baseline_type = "median"

In [ ]:
y_pred = baseline_model_predictor(data_test, baseline_type)

In [ ]:
y_pred.shape

In [ ]:
config.update({
        'baseline_type': baseline_type
    })


In [ ]:
# Input and output have the same shape
assert y_pred.shape == data_test.shape

In [ ]:
#| export utils
import tensorflow as tf
def get_windows_mse(predictions, original_data):
    " Function that calculates the mse for each of the windows in which an auto-encoder model has made a prediction."
    # Test that dimensions are correct.
    assert predictions.shape == original_data.shape
    # Create a mse object
    mse = tf.keras.losses.MeanSquaredError(
        reduction=tf.keras.losses.Reduction.NONE)
    # We need to adapt the axes to calculate the mse in the manner we want.
    prediction_swaped = np.swapaxes(predictions,1,2)
    original_data_swaped = np.swapaxes(original_data,1,2)
    # Calculate mses
    windows_mse = mse(original_data_swaped, prediction_swaped).numpy()
    
    return windows_mse

In [ ]:
%%time
windows_mse = get_windows_mse(y_pred,data_test)

In [ ]:
windows_mse.shape

MSE per time series

In [ ]:
windows_mse.mean(axis=0)

Total MSE:

In [ ]:
windows_mse.mean()

Log results on wandb:

In [ ]:
run_baseline.log({
    'mse_overall': windows_mse.mean(),
    'mse_signal': windows_mse.mean(axis=0), # MSE of each variable
    'mse_raw': windows_mse # Raw mse. It has the mse for each window and signal
})

Plot figure to visualize MSE per variable:

In [ ]:
import matplotlib.pyplot as plt

plt.bar(list(df_train.columns),  windows_mse.mean(axis=0), align='center', alpha=0.5)
plt.xticks(list(df_train.columns), rotation='vertical')
plt.ylabel('MSE')
plt.title('variable')


wandb.log({"mse_signal_plot": plt})

In [ ]:
run_baseline.finish()

It proves that `get_windows_mse()` is working correctly compared to the sklearn function. This check is done because the function is swapped, and it is convenient that the results are the same regardless of the method used.


In [ ]:
%%time
from sklearn.metrics import mean_squared_error
rme_sklearn = [0]* y_pred.shape[2]
for i in range(0,y_pred.shape[2]):
    y_pred_sel = y_pred[:,:,i]
    test_sel = data_test[:,:,i]
    rme_sklearn[i] = mean_squared_error(test_sel,y_pred_sel)

In [ ]:
rme_sklearn_round = np.around(rme_sklearn, decimals=3)
rme_keras_round = np.around(windows_mse.mean(axis=0),decimals=3)

In [ ]:
test.all_equal(rme_sklearn_round, rme_keras_round)